In [1]:
from pyspark.sql import *
from pyspark.ml.classification import LogisticRegression


In [2]:
trainDF=spark.read.parquet("/mnt/adls2/trainset.parquet/")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4428870687339165> in <module> () 
 ----> 1 trainDF = spark . read . parquet ( "/mnt/adls2/trainset.parquet/" ) 

 /databricks/spark/python/pyspark/sql/readwriter.pyc in parquet (self, *paths) 
 307 [ ( 'name' , 'string' ) , ( 'year' , 'int' ) , ( 'month' , 'int' ) , ( 'day' , 'int' ) ] 
 308 """
 --> 309 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 310 
 311 @ ignore_unicode_prefix 

 /databricks/spark/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1158 answer = self . gateway_client . send_command ( command ) 
 1159 return_value = get_return_value(
 -> 1160 answer, self.gateway_client, self.target_id, self.name)
 1161 
 1162 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.pyc in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.6-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 318 raise Py4JJavaError(
 319 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 320 format(target_id, ".", name), value)
 321 else : 
 322 raise Py4JError(

 Py4JJavaError : An error occurred while calling o302.parquet.
: com.microsoft.azure.datalake.store.ADLException: Error getting info for file /trainset.parquet
Error fetching access token
Operation null failed with exception java.net.MalformedURLException : no protocol: 72f988bf-86f1-41af-91ab-2d7cd011db47
Last encountered exception thrown after 5 tries. [java.net.MalformedURLException,java.net.MalformedURLException,java.net.MalformedURLException,java.net.MalformedURLException,java.net.MalformedURLException]
 [ServerRequestId:null]
	at com.microsoft.azure.datalake.store.ADLStoreClient.getExceptionFromResponse(ADLStoreClient.java:1169)
	at com.microsoft.azure.datalake.store.ADLStoreClient.getDirectoryEntry(ADLStoreClient.java:737)
	at com.microsoft.azure.datalake.store.ADLStoreClient.getDirectoryEntry(ADLStoreClient.java:718)
	at com.databricks.adl.AdlFileSystem.getFileStatus(AdlFileSystem.java:390)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1$$anonfun$apply$15.apply(DatabricksFileSystemV2.scala:643)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1$$anonfun$apply$15.apply(DatabricksFileSystemV2.scala:640)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:107)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1.apply(DatabricksFileSystemV2.scala:640)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1.apply(DatabricksFileSystemV2.scala:640)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:313)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:188)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:183)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:400)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:221)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:400)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:298)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:400)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.getFileStatus

In [3]:
lr = LogisticRegression(maxIter=100, regParam=0.01, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(trainDF)

# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

In [4]:
truePositiveRate

In [5]:
trainingSummary = lrModel.summary
roc = trainingSummary.truePositiveRateByLabel
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

In [6]:
truePositives=trainingSummary.truePositiveRateByLabel

In [7]:
display([list('0123456789'),truePositives].toDF())

In [8]:
testDF=spark.read.parquet('/mnt/adls/testset.parquet')

In [9]:
predictions=lrModel.transform(testDF)

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

In [11]:
!pip install Keras
#!pip install tensorflow
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K